In [ ]:
import pandas as pd
import numpy as np
import shared

ve_or_ju = "ju"


In [ ]:
def all_years_to_one_df(ve_or_ju):
    all = pd.DataFrame()
    for year in shared.years[ve_or_ju]:
        results = pd.read_csv(f'data/results_with_dist_j{year}_{ve_or_ju}.tsv', delimiter="\t")
        if(all.empty):
            all = results
        else:
            all = all.append(results)

        display(f'{year} {ve_or_ju} {all.shape}')
    return all        

In [ ]:
def group_by_team(ve_or_ju):
    all = all_years_to_one_df(ve_or_ju)

    all["log_leg_pace"] = np.log(all["leg-time"] / (60 * all["leg_distance"]))
    #all["disqualified_num"] = np.mean(all["disqualified"])
    by_team = all[["team-id", "team-name", "team-nro", "log_leg_pace", "disqualified"]].dropna().groupby(['team-name']).agg(["median", "mean", "std", "count", "sum"])
    #display(by_team.tail().round(2))
    
    flat = pd.DataFrame(by_team.to_records(), index=by_team.index)
    flat = pd.DataFrame(by_team["team-id"]["mean"]).reset_index()
    flat = flat.rename(columns={"mean": "team_id"})
    flat["team_num_mean"] = by_team["team-nro"]["mean"].values
    flat["log_leg_pace_mean"] = by_team["log_leg_pace"]["mean"].values
    flat["log_leg_pace_std"] = by_team["log_leg_pace"]["std"].values
    flat["legs"] = by_team["log_leg_pace"]["count"].values
    flat["disqualified_legs"] = by_team["disqualified"]["sum"].values
    #display(flat.head())
    
    return flat

In [ ]:
by_team = group_by_team(ve_or_ju)
by_team.head().round(2)

In [ ]:
by_team[by_team["disqualified_legs"] > 0]

In [ ]:
by_team[by_team["team-name"] == "Virastokeskuksen vikkelät"]

In [ ]:
# Export file used in other project (jukola-nlp)
by_team.to_csv(f'data/team_names_{ve_or_ju}.tsv', sep="\t")